In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install py7zr

In [ ]:
df = pd.read_csv("../input/cifar-10/trainLabels.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
import py7zr
archive = py7zr.SevenZipFile('../input/cifar-10/train.7z',mode = 'r')
archive.extractall(path="../temp")

In [ ]:
cd "../temp"

In [ ]:
%ls

In [ ]:
cd train

In [ ]:



%pwd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img

In [ ]:
im  = img.imread('1.png')

plt.imshow(im)
im.shape

In [ ]:
classes = df['label'].unique()

In [ ]:
classes

In [ ]:
train_images_path = "/kaggle/temp/train"
val_images_path = '/kaggle/temp/validation'

In [ ]:
os.mkdir(val_images_path)

In [ ]:
%pwd

In [ ]:
cd ..

In [ ]:
%ls

In [ ]:
cd train

In [ ]:
%pwd

In [ ]:
for c in classes:
    train_path  = os.path.join(train_images_path,c)
    
    if not os.path.exists(train_path):
        os.mkdir(train_path)
    validation_path = os.path.join(val_images_path,c)
    if not os.path.exists(validation_path):
        os.mkdir(validation_path)
        
    

In [ ]:
for ind in df.index:
    x = str(df['id'][ind]) + ".png"
    path1 = os.path.join(train_images_path,x)
    
    path2 = os.path.join(train_images_path,df['label'][ind],x)
    path3 = os.path.join(val_images_path,df['label'][ind],x)
    r = np.random.random()
    if r<=0.8:
        os.replace(path1,path2)
    else:
        os.replace(path1,path3)

In [ ]:
%pwd

In [ ]:
%ls

In [ ]:
cd frog

In [ ]:
%ls

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
%ls

In [ ]:
cd validation

In [ ]:
%ls

In [ ]:
cd ..

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
#                                   rotation_range=10,
#                                   zoom_range=0.1,
#                                   width_shift_range=0.1,
#                                   height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=False)


train_generator  = train_datagen.flow_from_directory(directory = r"./train/",target_size = (224,224),color_mode = 'rgb',batch_size = 32,
                                                     class_mode='categorical',shuffle= True,seed = 42)


val_datagen = ImageDataGenerator(featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
#                                   rotation_range=10,
#                                   zoom_range=0.1,
#                                   width_shift_range=0.1,
#                                   height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=False)


validation_generator  = val_datagen.flow_from_directory(directory = r"./validation/",target_size = (224,224),color_mode = 'rgb',batch_size = 32,
                                                     class_mode='categorical',shuffle= True,seed = 42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Activation,Flatten,Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical





In [ ]:
from keras.callbacks import ReduceLROnPlateau

reduceLROnPlateau  = ReduceLROnPlateau(monitor='val_accuracy',
                                     patience=3,
                                     verbose=1,
                                     factor=0.1,
                                     min_lr=0.00001)
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

In [ ]:
def mymodel():
    model=Sequential()
    model.add( Conv2D(filters=32, kernel_size=(7,7), strides=(1,1), padding='valid', activation=None, use_bias=False, input_shape=(224,224,3)) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add( Conv2D(filters=48, kernel_size=(7,7), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add( Conv2D(filters=64, kernel_size=(7,7), strides=(1,1), padding='valid', activation=None, use_bias=False) )
    model.add(BatchNormalization())
    model.add(Activation('relu'))

   
    model.add(Flatten())

    model.add(Dense(units=10))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model  = Sequential()
#     # first layer 


#     model.add(Conv2D(16,(3,3),input_shape=(32,32,3)))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization())

#     model.add(Conv2D(32,(3,3),input_shape=(32,32,3)))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization())


#     model.add(Conv2D(32,(3,3),input_shape=(32,32,3)))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))


#     model.add(MaxPooling2D(pool_size = (2,2)))


#     model.add(Conv2D(64,(3,3)))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.2))

#     model.add(Conv2D(64,(3,3)))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.2))

#     model.add(MaxPooling2D(pool_size = (2,2)))


#     model.add(Conv2D(128,(3,3)))
#     model.add(Activation("relu"))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.25))




#     model.add(MaxPooling2D(pool_size = (2,2)))

#     # model.add(Conv2D(256,(3,3)))
#     # model.add(Activation("relu"))
#     # model.add(BatchNormalization())
#     # model.add(Dropout(0.25))

#     # model.add(MaxPooling2D(pool_size = (2,2)))


#     model.add(Flatten())

#     model.add(Dense(128))
#     model.add(Activation("relu"))


#     # model.add(Dense(64))
#     # model.add(Activation("relu"))


#     model.add(Dense(10))
#     model.add(Activation("softmax"))
#     model.compile(loss="categorical_crossentropy",optimizer = 'adam',metrics=['accuracy'])
    return model

In [ ]:
my_mod = mymodel()

In [ ]:
tf.keras.utils.plot_model(my_mod,show_shapes=True)

In [ ]:
history = my_mod.fit(train_generator, epochs=10,  callbacks=[early_stopping_callback, reduceLROnPlateau],workers=8, use_multiprocessing=True, verbose=1, validation_data=validation_generator)

In [ ]:
%cd ..

In [ ]:
import py7zr

archive = py7zr.SevenZipFile('../kaggle/input/cifar-10/test.7z', mode='r')
archive.extractall(path='../temp')

In [ ]:
test_datagen = ImageDataGenerator(featurewise_center=False,
                                  samplewise_center=False,
                                  featurewise_std_normalization=False,
                                  samplewise_std_normalization=False,
                                  zca_whitening=False,
#                                   rotation_range=10,
#                                   zoom_range=0.1,
#                                   width_shift_range=0.1,
#                                   height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=False
                                  )

test_generator = test_datagen.flow_from_directory(
    directory=r"./test/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
label = my_mod.predict(X_test_reshaped)

In [ ]:
data= {
    'ImageId':list(range(1,28001)),
    'Label':label
}
# print(data)
df = pd.DataFrame(data).set_index('ImageId')
df